In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM,Activation,Dense,Dropout,Input,Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils.data_utils import pad_sequences
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline

In [ ]:
ds = pd.read_csv('/content/spam.csv',delimiter=',',encoding='latin-1')
ds.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [ ]:
x=ds.v2
y=ds.v1

le=LabelEncoder()
y=le.fit_transform(y)
y=y.reshape(-1,1)

In [ ]:
#splitting data
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.15,random_state=0)

In [ ]:
#splitting sentences into words through tokenization
#keras provides text_to_word_sequence() function to do so
max_words=1000
max_len=150
tok=Tokenizer(num_words=max_words)
tok.fit_on_texts(x_train)
sequences=tok.texts_to_sequences(x_train)
sequences_matrix=pad_sequences(sequences,maxlen=max_len)

In [ ]:
def RNN():
  inputs = Input(name='inputs',shape=[max_len])
  layer=Embedding(max_words,50,input_length=max_len)(inputs)
  layer=LSTM(64)(layer)
  layer=Dense(256,name='FC1')(layer)
  layer=Activation('relu')(layer)
  layer=Dropout(0.5)(layer)
  layer=Dense(1,name='out_layer')(layer)
  model=Model(inputs=inputs,outputs=layer)
  return model

In [ ]:
model=RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 150)]             0         
                                                                 
 embedding (Embedding)       (None, 150, 50)           50000     
                                                                 
 lstm (LSTM)                 (None, 64)                29440     
                                                                 
 FC1 (Dense)                 (None, 256)               16640     
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 1)                 257   

In [ ]:
model.fit(sequences_matrix,y_train,batch_size=120,epochs=10,validation_split=0.2,
          callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Epoch 1/10
32/32 [==============================] - 15s 346ms/step - loss: 0.2913 - accuracy: 0.9092 - val_loss: 0.1498 - val_accuracy: 0.9831
Epoch 2/10
32/32 [==============================] - 10s 306ms/step - loss: 0.0895 - accuracy: 0.9823 - val_loss: 0.0976 - val_accuracy: 0.9863
Epoch 3/10
32/32 [==============================] - 9s 277ms/step - loss: 0.0494 - accuracy: 0.9902 - val_loss: 0.1133 - val_accuracy: 0.9873


In [ ]:
test_sequences=tok.texts_to_sequences(x_test)
test_sequences_matrix=pad_sequences(test_sequences,maxlen=max_len)

In [ ]:
accuracy=model.evaluate(test_sequences_matrix,y_test)
accuracy

27/27 [==============================] - 2s 61ms/step - loss: 0.0819 - accuracy: 0.9856


[0.08186523616313934, 0.9856459498405457]

In [ ]:
#TESTING
testing_content=['olollololoolloloodfsdfdbsfhvvAJSjhvjeVJHVVdbJ,FBBbvsdvFHVSDBb']

txts=tok.texts_to_sequences(testing_content)
txts=pad_sequences(txts,maxlen=max_len)

In [ ]:
prediction=model.predict(txts)
if prediction<0.5:
  print('ham')
else:
  print('spam')

1/1 [==============================] - 0s 33ms/step
ham
